In [ ]:
import random

# Function to count occurrences of motifs in a sequence
def count_motifs(sequence, motif_length):
    motifs_count = {}
    for i in range(len(sequence) - motif_length + 1):
        motif = sequence[i:i + motif_length]
        motifs_count[motif] = motifs_count.get(motif, 0) + 1
    return motifs_count

# Function to find the most common motif and its count
def enumerate_most_common_motif(sequence, motif_length):
    motifs_count = count_motifs(sequence, motif_length)
    most_common_motif = max(motifs_count, key=motifs_count.get)
    return most_common_motif, motifs_count[most_common_motif]

# Function to generate a random motif from a sequence
def generate_random_motif(sequence, motif_length):
    start_index = random.randint(0, len(sequence) - motif_length)
    return sequence[start_index:start_index + motif_length]

# Function to score a motif against a target motif
def score_motif(motif, target_motif):
    score = 0
    for i in range(len(motif)):
        if motif[i] == target_motif[i]:
            score += 1
    return score

# Function to perform mutation on a motif
def mutate_motif(motif, motif_length):
    index = random.randint(0, motif_length - 1)
    nucleotides = ['A', 'T', 'C', 'G']
    new_nucleotide = random.choice(nucleotides)
    return motif[:index] + new_nucleotide + motif[index + 1:]

# Genetic algorithm for motif discovery//probabilitic
def genetic_algorithm(sequence, motif_length, population_size, generations):
    target_motif = generate_random_motif(sequence, motif_length)
    population = [generate_random_motif(sequence, motif_length) for _ in range(population_size)]
    for _ in range(generations):
        scores = [score_motif(motif, target_motif) for motif in population]
        best_index = scores.index(max(scores))
        best_motif = population[best_index]
        population = [best_motif] * population_size
        for i in range(population_size):
            for j in range(motif_length):
                if random.random() < 0.1:
                    population[i] = mutate_motif(population[i], motif_length)
    return best_motif

# Greedy algorithm for motif discovery
def greedy_motif_search(sequences, k, t):
    best_motifs = [seq[:k] for seq in sequences]
    for i in range(len(sequences[0]) - k + 1):
        motif = sequences[0][i:i + k]
        motifs = [motif]
        for j in range(1, t):
            profile = form_profile(motifs)
            next_motif = most_probable_kmer(sequences[j], k, profile)
            motifs.append(next_motif)
        if score_motifs(motifs) < score_motifs(best_motifs):
            best_motifs = motifs
    return best_motifs

# Function to form a profile matrix from given motifs
def form_profile(motifs):
    profile = {'A': [], 'C': [], 'G': [], 'T': []}
    for i in range(len(motifs[0])):
        column = [motif[i] for motif in motifs]
        for nucleotide in profile:
            profile[nucleotide].append(column.count(nucleotide) / len(column))
    return profile

# Function to find the most probable k-mer in a sequence given a profile
def most_probable_kmer(text, k, profile):
    max_prob = -1
    most_probable = ""
    for i in range(len(text) - k + 1):
        kmer = text[i:i + k]
        prob = 1
        for j in range(k):
            prob *= profile[kmer[j]][j]
        if prob > max_prob:
            max_prob = prob
            most_probable = kmer
    return most_probable

# Function to calculate the total score of a set of motifs
def score_motifs(motifs):
    score = 0
    for i in range(len(motifs[0])):
        column = [motif[i] for motif in motifs]
        most_common = max(set(column), key=column.count)
        score += sum(1 for nucleotide in column if nucleotide != most_common)
    return score

# Main function to test the motif discovery algorithms
def main():

    sequences = [
        "ATGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC",
        "ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG",
        "GCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA"
    ]

    motif_length = 21

    # Enumeration
    most_common_enumeration, count_enumeration = enumerate_most_common_motif("".join(sequences), motif_length)
    print("Most common motif using Enumeration:", most_common_enumeration)
    print("Number of occurrences:", count_enumeration)

    # Deterministic Optimization (Greedy Algorithm)
    best_motifs_greedy = greedy_motif_search(sequences, motif_length, len(sequences))
    most_common_greedy, count_greedy = enumerate_most_common_motif("".join(best_motifs_greedy), motif_length)
    print("\nMost common motif using Deterministic(Greedy) Algorithm:", most_common_greedy)
    print("Number of occurrences:", count_greedy)

    # Probabilistic Optimization (Genetic Algorithm)
    population_size = 10
    generations = 100
    best_motif_genetic = genetic_algorithm(sequences[0], motif_length, population_size, generations)
    print("\nMost common motif using Probabilistic Optimization(Genetic) Algorithm:", best_motif_genetic)

if __name__ == "__main__":
    main()


Most common motif using Enumeration: GCTAGCTAGCTAGCTAGCTAG
Number of occurrences: 20

Most common motif using Deterministic(Greedy) Algorithm: AGCTAGCTAGCTAGCTAGCTA
Number of occurrences: 2

Most common motif using Probabilistic Optimization(Genetic) Algorithm: TAGCTAGCTAGCTAGCTGGCT
